In [ ]:
%load_ext autoreload

In [2]:
"Results for k-medoids algorithm in table 6."

'Results for k-medoids algorithm in table 6.'

In [3]:
import numpy as np
import pickle

from basicOperations.manifoldOperations import unsupervised

In [4]:
numberSubjects = 91
numberChannels = 12
numberGestures = 10
trialsPerGesture = 36

In [5]:
collateData = []
LABELS = []

for subjectNumber in range(1, 100):
    if subjectNumber not in [5, 11, 14, 17, 18, 35, 38, 41]:
        name = "../formattedData/" + str(subjectNumber) + ".pkl"
        with open(name, "rb") as file:
            loadedData = pickle.load(file)

        Features = loadedData["EMG"]
        mean = np.mean(Features, axis = -1)
        std = np.std(Features, axis = -1)
        normalizedFeatures = (Features - mean[..., np.newaxis])/std[..., np.newaxis]
        Labels = loadedData["Labels"]
        samples = 2 * loadedData["Frequency"]

        gesturesLabels = np.zeros((numberGestures * trialsPerGesture, numberChannels, numberChannels))
        for i in range(numberGestures * trialsPerGesture):
            gesturesLabels[i] = (1/samples) * normalizedFeatures[i, :, :] @ normalizedFeatures[i, :, :].T
        
        collateData.append(gesturesLabels)
        LABELS.append(Labels)

In [6]:
collateData = np.array(collateData)
LABELS = np.array(LABELS)
print(collateData.shape)
print(LABELS.shape)

(91, 360, 12, 12)
(91, 360)


In [7]:
kMedoids = unsupervised()

In [8]:
corrects = np.zeros((numberSubjects))

In [9]:
for subject in range(numberSubjects):
    kmedoids = kMedoids.kMedoids(collateData[subject].reshape(-1, numberChannels, numberChannels), numberChannels, numberGestures)
    clusterLabels = kmedoids.labels_
    medoidIndices = kmedoids.medoid_indices_
    medoidLabels = LABELS[subject, medoidIndices]

    assignedLabels = np.zeros((numberGestures * trialsPerGesture))
    for i in range(numberGestures * trialsPerGesture):
        assignedLabels[i] = medoidLabels[clusterLabels[i]]
    corrects[subject] = np.mean(assignedLabels == LABELS[subject])

In [10]:
for i in range(numberSubjects):
    print("{:.3f}".format(corrects[i]))
print("{:.3f}".format(np.mean(corrects)))

0.503
0.747
0.486
0.867
0.378
0.708
0.192
0.578
0.408
0.511
0.361
0.631
0.181
0.900
0.558
0.861
0.525
0.358
0.650
0.183
0.822
0.797
0.925
0.661
0.972
0.475
0.367
0.592
0.797
0.983
0.714
0.908
0.797
0.589
0.411
0.197
0.589
0.828
0.783
0.261
0.672
0.172
0.281
0.417
0.767
0.739
0.867
0.894
0.711
0.711
0.794
0.753
0.739
0.619
0.622
0.931
0.725
0.644
0.497
0.758
0.853
0.347
0.611
0.717
0.717
0.703
0.914
0.258
0.533
0.828
0.678
0.986
0.714
0.939
0.475
0.608
0.600
0.739
0.625
0.875
0.739
0.933
0.736
0.556
0.700
0.358
0.706
0.728
0.761
0.231
0.581
0.639


In [11]:
print(np.mean(corrects))

0.6386446886446887
